In [7]:
import requests
import pandas as pd

In [ ]:
indicators = {
    'CO2_pc': ('https://api.ourworldindata.org/v1/indicators/944153.data.json', 'https://api.ourworldindata.org/v1/indicators/944153.metadata.json'),
    'CO2_share': ('https://api.ourworldindata.org/v1/indicators/944147.data.json', 'https://api.ourworldindata.org/v1/indicators/944147.metadata.json'),
    'GHG_annual': ('https://api.ourworldindata.org/v1/indicators/899063.data.json', 'https://api.ourworldindata.org/v1/indicators/899063.metadata.json'),
    'CO2_cumulative': ('https://api.ourworldindata.org/v1/indicators/944121.data.json', 'https://api.ourworldindata.org/v1/indicators/944121.metadata.json'),
    'N2O_annual': ('https://api.ourworldindata.org/v1/indicators/899060.data.json', 'https://api.ourworldindata.org/v1/indicators/899060.metadata.json'),
    'CH4_annual': ('https://api.ourworldindata.org/v1/indicators/899057.data.json', 'https://api.ourworldindata.org/v1/indicators/899057.metadata.json'),
    'CO2_coal': ('https://api.ourworldindata.org/v1/indicators/944128.data.json', 'https://api.ourworldindata.org/v1/indicators/944128.metadata.json'),
    'CO2_oil': ('https://api.ourworldindata.org/v1/indicators/944140.data.json', 'https://api.ourworldindata.org/v1/indicators/944140.metadata.json'),
    'CO2_gas': ('https://api.ourworldindata.org/v1/indicators/944134.data.json', 'https://api.ourworldindata.org/v1/indicators/944134.metadata.json'),
    'CO2_flaring': ('https://api.ourworldindata.org/v1/indicators/944131.data.json', 'https://api.ourworldindata.org/v1/indicators/944131.metadata.json'),
    'CO2_transport': ('https://api.ourworldindata.org/v1/indicators/815831.data.json', 'https://api.ourworldindata.org/v1/indicators/815831.metadata.json'),
}


In [ ]:
# Lấy metadata
metadata_url = 'https://api.ourworldindata.org/v1/indicators/944153.metadata.json'
metadata_response = requests.get(metadata_url)
metadata = metadata_response.json()

entities = metadata['dimensions']['entities']['values']
entity_id_to_name = {entity['id']: entity['name'] for entity in entities if entity['code'] is not None}


In [ ]:
# Lấy dữ liệu cho một chỉ số
def fetch_indicator_data(indicator_name, url, entity_id_to_name):
    response = requests.get(url)
    data = response.json()
    
    values = data['values']
    years = data['years']
    entities = data['entities']
    
    records = []
    for i, value in enumerate(values):
        entity_id = entities[i]
        year_index = i % len(years)
        year = years[year_index]
        if 2010 <= year <= 2022 and entity_id in entity_id_to_name:
            country = entity_id_to_name[entity_id]
            records.append([country, year, value])
    
    return pd.DataFrame(records, columns=['Country', 'Year', indicator_name])


In [ ]:
df_final = pd.DataFrame()

# Hợp nhất dữ liệu
for indicator, (data_url, _) in indicators.items():
    df_indicator = fetch_indicator_data(indicator, data_url, entity_id_to_name)
    
    if df_final.empty:
        df_final = df_indicator
    else:
        df_final = pd.merge(df_final, df_indicator, on=['Country', 'Year'], how='outer')


In [ ]:
df_final.to_csv('full_data.csv', index=False)

print(df_final.head())

       Country  Year    CO2_pc  CO2_share  GHG_annual  CO2_cumulative  \
0  Afghanistan  2010  0.296733   0.025115  31517414.0       103516320   
1  Afghanistan  2011  0.404740   0.034377  35598764.0       115354630   
2  Afghanistan  2012  0.329389   0.028725  33642076.0       125389950   
3  Afghanistan  2013  0.293283   0.026256  32901880.0       134640460   
4  Afghanistan  2014  0.280299   0.025856  33513274.0       143810770   

   N2O_annual  CH4_annual   CO2_coal    CO2_oil   CO2_gas  CO2_flaring  \
0   4313534.5  17632924.0  2246032.0  5833088.0  271136.0          0.0   
1   4477510.5  18206366.0  4180624.0  7335328.0  307776.0          0.0   
2   4489223.0  18100962.0  3125392.0  6573216.0  307776.0          0.0   
3   4488093.0  18211250.0  3326912.0  5591264.0  296784.0          0.0   
4   4753052.0  18687494.0  3705783.0  5164638.0  271244.0          0.0   

   CO2_transport  
0      5540000.0  
1      6710000.0  
2      5850000.0  
3      4330000.0  
4      3530000.0  
